In [17]:
import pandas as pd
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import END, Graph
from typing import TypedDict, Dict
import re

In [ ]:
# Initialize LLM
llm = Ollama(model="llama3.1:8b", temperature=0.3)

In [19]:
# Load and clean data
def load_excel_data(file_path):
    df = pd.read_excel(file_path, sheet_name='Sheet1', header=None)
    headers = [str(col) if not pd.isna(col) else f"unnamed_{i}" 
               for i, col in enumerate(df.iloc[0])]
    df.columns = headers
    df = df.iloc[1:].reset_index(drop=True)
    return df

data = load_excel_data("Sales Report.xlsx")

In [20]:
# Define the single tool we need
def get_product_total_sales(state: Dict) -> Dict:
    """Calculate total sales for a specific product."""
    question = state['question']
    product_name = None
    
    # Simple product name extraction
    for product in data['Product Name']:
        if str(product).lower() in question.lower():
            product_name = product
            break
    
    if not product_name:
        return {"result": "Product not found in the data."}
    
    # Calculate total sales (sum all numeric columns except the first)
    sales_columns = [col for col in data.columns if col != 'Product Name']
    product_data = data[data['Product Name'] == product_name]
    
    if product_data.empty:
        return {"result": "No sales data available for this product."}
    
    total_sales = product_data[sales_columns].sum(axis=1).iloc[0]
    return {"result": f"Total sales for {product_name}: {total_sales:.2f}"}


In [21]:
# Simplified state
class AgentState(TypedDict):
    question: str
    result: str

In [22]:
# Build the minimal graph
workflow = Graph()

workflow.add_node("get_sales", get_product_total_sales)
workflow.add_node("format_response", lambda state: {"response": state['result']})

workflow.add_edge("get_sales", "format_response")
workflow.add_edge("format_response", END)

workflow.set_entry_point("get_sales")
app = workflow.compile()

In [23]:
# Simplified agent runner
def run_agent(question: str) -> str:
    result = app.invoke({"question": question})
    return result.get("response", "No response generated")


In [25]:
print(run_agent("Show me total sales for PC-1000"))


Total sales for PC-1000: 108.62


In [26]:
print(run_agent("How much did Solar sell in total?"))

Total sales for Solar: 312.30
